In [1]:
import netgen.gui
from ngsolve import *
import numpy as np

from ngsolve.solvers import CG

In [2]:
from netgen.geom2d import unit_square
mesh = Mesh(unit_square.GenerateMesh(maxh=0.05))
Draw (mesh)

In [3]:
fes = H1(mesh, order=6,dirichlet=".*")
gfu_a = GridFunction(fes,name = "u_a")
gfu_b = GridFunction(fes,name = "u_b")

In [9]:
ua = fes.TrialFunction()
va = fes.TestFunction()

Aa = BilinearForm(fes)
Fa = LinearForm(fes)

Aa += ua*va*dx

Fa += va*dx

Aa.Assemble()
Fa.Assemble()

c = Preconditioner(Aa,"local",test = True)
c.Update()

#gfu_a.vec.data = Aa.mat.Inverse(fes.FreeDofs()) * Fa.vec

with TaskManager():
    CG (mat=Aa.mat, pre=c.mat, rhs=Fa.vec, sol=gfu_a.vec,printrates=False, maxsteps=200,tol=1e-6)
Draw(gfu_a)

#Draw(gfu_a)

In [5]:
ub = fes.TrialFunction()
vb = fes.TestFunction()

Ab = BilinearForm(fes)
Fb = LinearForm(fes)

Ab += grad(ub)*grad(vb)*dx

Fb += vb*dx

Ab.Assemble()
Fb.Assemble()

gfu_b.vec.data = Ab.mat.Inverse(fes.FreeDofs()) * Fb.vec

Draw(gfu_b)